# v3t3p2ph2_pso_gene.ipynb

Hyperparameter search on the v3_from_t3p2_ph2 model using either Particle Swarm Optimization, a genetic algorithm, or a combinatin of both using two machines.

When running both algorithms, one machine maintains a population evolved through the PSO algorithm, whereas the second machine uses a genetic algorithm. They share their configurations through a shared file system.

-Sergio Verduzco-Flores  
June 2021

In [1]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from datetime import datetime
from multiprocessing import Pool
from pathos.multiprocessing import ProcessingPool

/home/z/projects/draculab


In [2]:
%load_ext Cython

In [3]:
%%cython
from draculab import *

In [4]:
%cd /home/z/projects/draculab/notebook/spinal/
from v3ft3p2ph2_net import *
%cd ../..

/home/z/projects/draculab/notebook/spinal
/home/z/projects/draculab


In [5]:
# Utility methods, ranges, search configuration parameters

focus_params = False # whether to focus mutations on a specific set of parameters
max_evals = 8 # maximum number of times to evaluate a configuration
target_fitness = 0.02 # stop if fitness reaches this value

# Specify paramters and ranges
# defaults are based on incumbent 9 of smac_pars in v3_normal_smac_test.ipynb
ranges = {"A__M_lrate" : {"low": 0.1, "high": 20., "default":8.},
          "A__M_w_max_frac" : {"low": .05, "high": 1., "default":.4},
          "A__M_w_sum" : {"low": 1., "high": 40., "default":2.},
          "AL_thresh" : {"low": -.1, "high": 1., "default":.55},
          "b_e": {"low": .5, "high": 10., "default":1.},
          "C__C_antag": {"low": 0.1, "high": 3., "default":1.5},
          "C__C_p_antag": {"low": 0., "high": 1.5, "default":.25},
          "C__C_p_syne": {"low": 0., "high": 1., "default":.3},
          "C__C_syne": {"low": 0., "high": 2., "default":1.},
          "C_adapt_amp": {"low": 0., "high": 15., "default":4.},
          "C_cid" : {"low": 0.1, "high": 2., "default":.14},
          "C_sigma" : {"low": 0., "high": 1., "default":.3},
          "C_slope" : {"low": 0.5, "high": 4., "default":3.},
          "C_tau" : {"low": 0.01, "high": .4, "default":.2},
          "C_tau_slow" : {"low": 2., "high": 40., "default":10.},
          "C_thresh" : {"low": -0.2, "high": 1.6, "default":.7},
          "CE__CI_w": {"low": 0., "high": 2.5, "default":.5},
          "CI__CE_w": {"low": -2.5, "high": 0, "default":-1.3},
          "g_e_03": {"low": 15., "high": 25., "default":20.},  
          "CI_slope" : {"low": 1., "high": 5., "default":3.5},
          "CI_tau" : {"low": 0.01, "high": .2, "default":.1},
          "CI_thresh" : {"low": 0., "high": 2., "default":1.4},  
          "g_e_factor": {"low": 0.5, "high": 4., "default":3.},
          "II_g_03": {"low": 2., "high": 9., "default":3.},
          "M__C_lrate" : {"low": 5., "high": 500., "default":300.},
          "M__C_w_sum": {"low": 0.5, "high": 8., "default": 2.3},
          "M__M_w": {"low": 0., "high": -3., "default":-.5},
          "M_cid": {"low": 0.05, "high": 2., "default": .25},
          "M_des_out_w_abs_sum": {"low": 0.5, "high": 4., "default": 2.},
          "M_tau": {"low": .005, "high":.2, "default":.01},
          "SF_thresh_03": {"low": .2, "high": 1.1, "default":.4},
          "SPF__SPF_w": {"low": -3., "high": 0., "default":-1.5}
         }


#par_list = [name for name in ranges] # ordered list with names of the parameters
par_list = list(ranges.keys())
# parameters to focus on
main_pars = ["A__M_w_max_frac", "A__M_w_sum", "A__M_lrate", "C_cid", "g_e_03",
             "C_sigma", "II_g_03", "SF_thresh_03", "M_cid", "M_tau", "M_des_out_w_abs_sum",
             "C_slope", "C_thresh", "C_tau","CI_slope", "CI_thresh", "CI_tau", "C_tau_slow"]
if focus_params:
    par_list = main_pars

In [6]:
# Functions for the genetic algorithm and for creating initial parameters

def mutate(cfg, name_list=par_list):
    """ Mutate a single parameter of the given configuration. 
    
        Args:
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] =  l + (h-l)*np.random.random()
    
def soft_mutate(cfg, ma, name_list=par_list):
    """ Soft-mutate a single parameter of the given configuration.
    
        A 'soft mutation' keeps the mutated value close to the original value.
        The maximum amplitude of the mutation is given by the 'ma' argument.
        
        Args:
            ma : float in (0,1]. Max. amplitude as fraction of the parameter's range 
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] = max(l, min(h, cfg[par_name] + ma * (h-l) * (np.random.random()-0.5)))

In [7]:
# Create the initial population: method 1

# create variations of the parameters we want to investigtate
pop_size = 90 # number of configurations in the population
pop = [{} for _ in range(pop_size)]
# first fill the default values in n_def elements
n_def = 3
for ind in range(n_def):
    for name in par_list:
        pop[ind][name] = ranges[name]['default']
        
# fill the rest with variations in both directions
chg_name = ["low", "default", "high"] # auxiliary list
for ind in range(n_def, pop_size):
    chg_dirs = [chg_name[i] for i in np.random.randint(3, size=len(par_list))]
    for idx, name in enumerate(par_list):
        pop[ind][name] = 0.5 * (ranges[name][chg_dirs[idx]] + ranges[name]["default"])
        
# Throw n_muts mutations, and n_soft_muts soft mutations
n_muts = 20
n_soft_muts = 10 # must have n_muts + n_soft_muts < pop_size-1
perm = np.random.permutation(range(1,pop_size)) # don't mutate the first element
for i in range(n_muts):
    mutate(pop[perm[i]])
for i in range(n_muts,n_muts+n_soft_muts):
    soft_mutate(pop[perm[i]], 0.2)

In [8]:
# Create the initial population: method 2

# Load the results from a previous run
#fname = '/home/z/projects/draculab/saves/v3ft3p2ph2_pop_2021-05-28__10_53'
fname = '/home/z/Dropbox (OIST)/saves/gene_2021-06-17'
with (open(fname, "rb")) as f:
    prev_pop1 = pickle.load(f)
    f.close()

fname = '/home/z/Dropbox (OIST)/saves/gene_2021-06-24'
with (open(fname, "rb")) as f:
    prev_pop2 = pickle.load(f)
    f.close()

fname = '/home/z/Dropbox (OIST)/saves/pso_2021-06-28'
with (open(fname, "rb")) as f:
    prev_pop3 = pickle.load(f)
    f.close()

fname = '/home/z/Dropbox (OIST)/saves/pso_2021-06-24'
with (open(fname, "rb")) as f:
    prev_pop4 = pickle.load(f)
    f.close()
    
fname = '/home/z/Dropbox (OIST)/saves/gene_2021-07-19'
with (open(fname, "rb")) as f:
    prev_pop5 = pickle.load(f)
    f.close()
    
fname = '/home/z/Dropbox (OIST)/saves/pso_2021-07-19'
with (open(fname, "rb")) as f:
    prev_pop6 = pickle.load(f)
    f.close()
# If the results are from a run with fewer parameters
# fill it with the default values.
for cfg in prev_pop1 + prev_pop2 + prev_pop3 + prev_pop4:
    for name in ranges:
        if not name in cfg:
            cfg[name] = ranges[name]['default']


In [9]:
# Mix configurations from the two methods
pop[0:15] = prev_pop1[:20]
pop[15:30] = prev_pop2[:20]
pop[30:45] = prev_pop3[:20]
pop[45:60] = prev_pop4[:20]
pop[60:75] = prev_pop5[:20]
pop[75:90] = prev_pop6[:20]

# Best configuration from June 11th
cfg11 = {'A__M_w_max_frac': 0.9,
         'A__M_w_sum': 2.0,
         'C_adapt_amp': 4.4,
         'C_cid': 0.12,
         'C_sigma': 0.67,
         'M_cid': 1.1,
         'M_des_out_w_abs_sum': 3.0,
         'g_e_factor': 3.0,
         'C_slope': 2.,
         'C_thresh': 1.1,
         'C_tau': 0.26,
         'C_tau_slow': 10.0,
         'A__M_lrate': 8.0,
         'AL_thresh': 0.55,
         'b_e': 1.0,
         'C__C_antag': 1.5,
         'C__C_p_antag': 0.25,
         'C__C_p_syne': 0.3,
         'C__C_syne': 1.0,
         'CE__CI_w': 0.5,
         'CI__CE_w': -1.3,
         'M__C_lrate': 300.0,
         'M__C_w_sum': 2.3,
         'M__M_w': -0.5,
         'SPF__SPF_w': -1.5,
         'fitness': None,
         'n_evals': 0,
         't_pres': 40,
         'par_heter': 0.01,
         'CI_slope': 2.5,
         'CI_thresh': 1.4,
         'CI_tau': 0.15,
         'g_e_03': 20.0,
         'II_g_03': 3.0,
         'M_tau': 0.01,
         'SF_thresh_03': 0.4}
cfg_syne={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.3,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.55,
     'b_e': 1.5,
     'C__C_antag': 1.5,
     'C__C_p_antag': 0.25,
     'C__C_p_syne': 0.28,
     'C__C_syne': 1.,
     'C_adapt_amp': 0.4,
     'C_cid': 0.15,
     'C_sigma': 0.5,
     'C_slope': 2.1,
     'C_tau': 0.23,
     'C_tau_slow': 40.0,
     'C_thresh': 0.93,
     'CE__CI_w': 0.5,
     'CI__CE_w': -1.3,
     'g_e_03': 25.,
     'CI_slope': 3.6,
     'CI_tau': 0.017,
     'CI_thresh': 1.4,
     'g_e_factor': 3.2,
     'II_g_03': 3.16,
     'M__C_lrate': 200.,
     'M__C_w_sum': 2.5,
     'M__M_w': 0.0,
     'M_cid': 1.1,
     'M_des_out_w_abs_sum': 3.,
     'M_tau': 0.024,
     'SF_thresh_03': 0.63,
     'SPF__SPF_w': -1.5,
     'fitness': None,
     'n_evals': 0,
     't_pres': 40.,
     'par_heter': 0.01}

cfg_std={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.34,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.56,
     'b_e': 1.,
     'C__C_antag': 1.6,
     'C__C_p_antag': 0.15,
     'C__C_p_syne': 0.26,
     'C__C_syne': 1.1,
     'C_adapt_amp': 0.0,
     'C_cid': 0.17,
     'C_sigma': 0.5,
     'C_slope': 2.25,
     'C_tau': 0.24,
     'C_tau_slow': 2.0,
     'C_thresh': 1.14,
     'CE__CI_w': 0.39,
     'CI__CE_w': -1.8,
     'g_e_03': 20.,
     'CI_slope': 3.9,
     'CI_tau': 0.06,
     'CI_thresh': 1.37,
     'g_e_factor': 3.,
     'II_g_03': 2.73,
     'M__C_lrate': 500.0,
     'M__C_w_sum': 3.28,
     'M__M_w': 0.0,
     'M_cid': 1.,
     'M_des_out_w_abs_sum': 1.87,
     'M_tau': 0.012,
     'SF_thresh_03': 0.59,
     'SPF__SPF_w': -1.6,
     'fitness': None,
     'n_evals': 0,
     't_pres': 40.,
     'par_heter': 0.01}

pop[68] = cfg_std
pop[78] = cfg_syne
pop[88] = cfg11

# If configurations have fewer parameters, fill them with the default values.
# Also, homogeneize parameters not in par_list
for cfg in pop:
    for name in ranges:
        if (not name in cfg or 
            (focus_params and not name in par_list)):
            cfg[name] = cfg11[name]

In [10]:
# reset fitness and number of evaluations
for dic in pop:
    if not 'fitness' in dic or not 'nevals' in dic:
        dic['fitness'] = None # average fitness value
        dic['n_evals'] = 0  # number of times fitness has been evaluated

In [11]:
# Set search parameters present in the configurations
for cfg in pop:
    cfg['t_pres'] = 40
    cfg['par_heter'] = 0.01

In [12]:
# print used configuration
for dic in pop[0:3]:
    print('{',end='')
    for name in dic.keys():
        print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name in net_conf:
#             print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name != 'fitness' or dic['fitness'] != None:
#             print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'A__M_w_max_frac':0.225, 'A__M_w_sum':2.0, 'C_adapt_amp':4.4, 'C_cid':0.14928385685463597, 'C_sigma':0.6235117516563925, 'M_cid':1.069072715299579, 'M_des_out_w_abs_sum':3.107279886064089, 'g_e_factor':3.0, 'C_slope':2.2675408763677987, 'C_thresh':0.8919820536944196, 'C_tau':0.26003435786600343, 'C_tau_slow':21.0, 'A__M_lrate':8.0, 'AL_thresh':0.55, 'b_e':1.0, 'C__C_antag':1.5, 'C__C_p_antag':0.25, 'C__C_p_syne':0.3, 'C__C_syne':1.0, 'CE__CI_w':0.5, 'CI__CE_w':-1.3, 'M__C_lrate':300.0, 'M__C_w_sum':2.3, 'M__M_w':-0.5, 'SPF__SPF_w':-1.5, 'fitness':None, 'n_evals':0, 't_pres':40, 'par_heter':0.01, 'g_e_03':20.0, 'CI_slope':4.158365408276022, 'CI_tau':0.09203223283805521, 'CI_thresh':1.4, 'II_g_03':6.4857818119346735, 'M_tau':0.01, 'SF_thresh_03':0.4, }

{'A__M_w_max_frac':0.225, 'A__M_w_sum':2.0, 'C_adapt_amp':4.4, 'C_cid':0.14928385685463597, 'C_sigma':0.6235117516563925, 'M_cid':1.069072715299579, 'M_des_out_w_abs_sum':3.107279886064089, 'g_e_factor':3.0, 'C_slope':1.9032393124940064,

{'A__M_w_max_frac': 0.225,
 'A__M_w_sum': 2.0,
 'C_adapt_amp': 4.4,
 'C_cid': 0.14928385685463597,
 'C_sigma': 0.6235117516563925,
 'M_cid': 1.069072715299579,
 'M_des_out_w_abs_sum': 3.107279886064089,
 'g_e_factor': 3.0,
 'C_slope': 2.2675408763677987,
 'C_thresh': 0.8919820536944196,
 'C_tau': 0.26003435786600343,
 'C_tau_slow': 21.0,
 'A__M_lrate': 8.0,
 'AL_thresh': 0.55,
 'b_e': 1.0,
 'C__C_antag': 1.5,
 'C__C_p_antag': 0.25,
 'C__C_p_syne': 0.3,
 'C__C_syne': 1.0,
 'CE__CI_w': 0.5,
 'CI__CE_w': -1.3,
 'M__C_lrate': 300.0,
 'M__C_w_sum': 2.3,
 'M__M_w': -0.5,
 'SPF__SPF_w': -1.5,
 'fitness': None,
 'n_evals': 0,
 't_pres': 40,
 'par_heter': 0.01,
 'g_e_03': 20.0,
 'CI_slope': 4.158365408276022,
 'CI_tau': 0.09203223283805521,
 'CI_thresh': 1.4,
 'II_g_03': 6.4857818119346735,
 'M_tau': 0.01,
 'SF_thresh_03': 0.4}

In [13]:
# A function that evaluates the fitness of a given configuration
def eval_config(cfg):
    """ Returns the error for a network with a given configuration.

        Args:
            cfg : a configuration dictionary.
        Returns:
            error : A float calculated from the sum of activities in the SPF layer.
    """
    np.random.seed() # will try to get a seed from /dev/urandom
    if cfg['n_evals'] > max_evals: # if the fitness has been evaluated "enough" times. See cell below...
        return cfg['fitness']
    t_pres = cfg['t_pres']
    
    # obtain a network with the given configuration
    net, pops_dict, hand_coords, m_idxs, pds = net_from_cfg(cfg,
    #net, pops_dict, hand_coords, m_idxs, pds = syne_net(cfg,
               t_pres = cfg['t_pres'], 
               par_heter = cfg['par_heter'],
               set_C_delay = False,
               rand_targets = True,
               C_noise = True,
               track_weights = False,
               track_ips = False,
               rot_SPF = False)
    # run the network
    run_time = 1000.
    #start_time = time.time()
    times, data, plant_data  = net.flat_run(run_time)
    #print('Execution time is %s seconds' % (time.time() - start_time))

    # calculate average error in last half of reaching
    P = pops_dict['P']
    arm_activs = plant_data[P]
    plant = net.plants[P]
    # modified copy-paste of plt.upd_ip_impl
    q1 = arm_activs[:,0]
    q2 = arm_activs[:,2]
    q12 = q1+q2
    c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
    c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                    c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
    coord_idxs = np.floor(times/t_pres).astype(int)
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time

    error_time = run_time - round(run_time/2.)
    error_idx = int(round(error_time/net.min_delay))
    hand_error = np.linalg.norm(c_hand-des_coords, axis=1)
    hand_error_integ = hand_error[error_idx:].sum()
    avg_hand_error = hand_error_integ / (hand_error.size - error_idx)

    #return hand_error_integ
    return avg_hand_error

---
# Genetic algorithm  

---

In [ ]:
# A function to produce offspring by crossing individuals
par_names = list(pop[0].keys()) # list with all parameter names

def create_offspring(cfg1, cfg2, par_list=par_names):
    """ Given 2 configurations, return 2 offspring from random swapping.
    
        To produce offspring, first we choose one split point in the
        dictionary. The first offspring has the values of cfg1 up to that
        point, and cfg2 afterwards. The second offspring has the cfg2 values
        up to the split point, and cfg1 afterwards. Since the dictionaries are
        not ordered, we use a parameter list to set the split point.
    
        Args:
            cfg1, cfg2: parameter dictionaries
            par_list: list with the keys in cfg1, cfg2
        Returns:
            cfg3, cfg4: dictionaries from swapping values in cfg1, cfg2
    """
    if focus_params:
        par_list = main_pars
    sp = np.random.randint(len(par_list))# split point as an index to par_list
    cfg3 = cfg1.copy()
    cfg4 = cfg2.copy()
    for i in range(sp, len(par_list)):
        cfg3[par_list[i]] = cfg2[par_list[i]]
        cfg4[par_list[i]] = cfg1[par_list[i]]
    return cfg3, cfg4

In [ ]:
####################################
###### The genetic algorithm ######
####################################

#pop = pop[0:15] # limit pop size for debugging
n_mates = 30 # number of individuals to mate at each generation (even number)
max_gens = 80 # maximum number of generations
n_soft_mut = 10 # number of individuals to soft-mutate per generation
r_soft_mut = 0.2 # relative amplitude of soft mutations
n_mut = 8 # number of individuals to mutate per generation
n_procs = 30 # number of processes to use for fitness evaluation
n_save = 3 # number of individuals to protect from replacement and mutation
use_pso = True # whether to insert configurations from the PSO algorithm
repl_num = 8 # how many individuals to replace with PSO configurations

# setting name for file where parameters will be stored
path = "/home/z/Dropbox (OIST)/saves/"
#path = "/home/z/projects/draculab/saves/"
fname1 = "gene"
fname2 = datetime.now().strftime('%Y-%m-%d')
#fname2 = '2021-07-14'
fname = path + fname1 + "_" + fname2

pop_len = len(pop)

for gen in range(max_gens):
    start_time = time.time()
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
    ######## parallel version, python multiprocessing module
#     with Pool(n_procs) as p:
#         fits = list(p.map(eval_config, pop))
#         p.close()
#         p.join()
    ######## parallel version, pathos
    with ProcessingPool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
    #print(fits)
    # 1.2) update the average fitness values
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        #print(nr)
        #print(fits[idx])
        #print(cfg['fitness'])
        if nr > 0:
            if nr <= max_evals:
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
    #---------------------------------------------------------------------------------
    # 1.3) Inserting configurations from the PSO algorithm
    if use_pso:
        pso_name = path + "pso_" + fname2
        try:
            with open(pso_name, 'rb') as f:
                pso_pop = pickle.load(f)
                f.close()
            pop[-repl_num:] = pso_pop[:repl_num]
            ext_pop = pop + pso_pop[repl_num:]
            print("Mixed pso and gene pops!!!")
        except IOError:
            if gen > 2: # if the pso algorithm should likely be done
                from warnings import warn
                warn('population ' + pso_name + 'could not be imported',
                     UserWarning)
            ext_pop = pop
    #---------------------------------------------------------------------------------
    # 2) Sort according to fitness. Lowest error first.
    if use_pso:
        pop = sorted(ext_pop, key=lambda d: d['fitness'])[:pop_len]
    else:
        pop = sorted(pop, key=lambda d: d['fitness'])
    # 2.1) Save current generation
    with open(fname, 'wb') as f:
        pickle.dump(pop, f)
        f.close()
    # 2.2) A quick message
    print("Generation %d evaluated. Best fitness: %.3f"%(gen,pop[0]['fitness']))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if pop[0]['fitness'] < target_fitness:
        print("Good enough parameters found. Stopping search.")
        break
    # 3) mate and replace
    # 3.1) Select individuals to be replaced with probability proportional to error
    fits.sort() # sort the fitnesses (now in the same order as pop)
    fits = np.array(fits)
    fits = fits/fits.sum() # normalize fitnesses so they add to 1
    cumsum_fits = fits[:] # cumsum_fits[i] = sum(fits[:i])
    for i in range(1,len(cumsum_fits)):
        cumsum_fits[i] = cumsum_fits[i-1] + cumsum_fits[i]
    repl_list = [] # list with indexes of individuals to be replaced
    while len(repl_list) < n_mates:
        min_r = cumsum_fits[n_save] # don't replace the first n_save individuals
        r = min_r + (1.-min_r) * np.random.random()
        candidate = n_save
        for i in range(n_save, len(fits)):
            if cumsum_fits[i] > r:
                break
            candidate += 1
        if candidate in repl_list:
            continue
        else:
            repl_list.append(candidate)
    print("to replace: ", end='')
    print(repl_list)
    # 3.2) Arrange individuals in random pairs
    perm = np.random.permutation(n_mates) # this will do 
    # 3.3) mate
    new_pops = []
    for i in range(int(np.floor(n_mates/2))):
        off1, off2 = create_offspring(pop[perm[2*i]], pop[perm[2*i+1]])
        new_pops.append(off1)
        new_pops.append(off2)
    # 3.4) replace
    for i, cfg in enumerate(new_pops):
        pop[repl_list[i]] = cfg
    # 4) mutate
    # 4.1) soft mutations
    for _ in range(n_soft_mut):
        idx = np.random.randint(len(pop))
        if idx < n_save:
            copy = pop[idx].copy()
            soft_mutate(copy, r_soft_mut)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            soft_mutate(pop[idx], r_soft_mut)
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
    # 4.2) mutations
    # 4.2.1) select individuals to mutate
    # sq_fits = fits*fits
    #cumsum_sq_fits = fits * fits # cumsum_sq_fits[i] = sum(sq_fits[:i])
    cumsum_sq_fits = fits # proportional to fits, rather than its square
    cumsum_sq_fits = cumsum_sq_fits / cumsum_sq_fits.sum()
    for i in range(1,len(cumsum_sq_fits)):
        cumsum_sq_fits[i] = cumsum_sq_fits[i-1] + cumsum_sq_fits[i]
    mut_list = [] # list with indexes of individuals to be mutate
    while len(mut_list) < n_mut:
        r = np.random.random()
        candidate = 0
        for i in range(len(fits)):
            if cumsum_sq_fits[i] > r:
                break
            candidate += 1
        if candidate in mut_list:
            continue
        else:
            mut_list.append(candidate)
    print("to mutate: ", end='')
    print(mut_list)
    for idx in mut_list:
        if idx < n_save:
            copy = pop[idx].copy()
            mutate(copy)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            mutate(pop[idx])
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
            
    print('generation %d finished in %s seconds' % (gen, time.time() - start_time))

    

Generation 0 evaluated. Best fitness: 0.039
Mean fitness = 0.122
to replace: [104, 84, 105, 46, 80, 98, 21, 109, 93, 86, 103, 71, 54, 89, 91, 12, 85, 24, 112, 110, 99, 51, 115, 107, 60, 111, 108, 48, 81, 58]
to mutate: [65, 108, 105, 87, 62, 92, 42, 86]
generation 0 finished in 14973.354380369186 seconds
Generation 1 evaluated. Best fitness: 0.039
Mean fitness = 0.101
to replace: [83, 72, 115, 74, 80, 53, 86, 95, 24, 55, 114, 98, 111, 106, 21, 100, 88, 51, 10, 76, 79, 75, 104, 78, 65, 92, 54, 26, 62, 77]
to mutate: [97, 114, 72, 104, 96, 91, 90, 87]
generation 1 finished in 15009.575298070908 seconds
Generation 2 evaluated. Best fitness: 0.041
Mean fitness = 0.089
to replace: [104, 60, 63, 75, 53, 98, 94, 5, 33, 112, 88, 105, 59, 115, 51, 81, 31, 114, 14, 79, 29, 64, 77, 17, 26, 8, 109, 24, 113, 73]
to mutate: [79, 78, 108, 63, 61, 83, 81, 105]
generation 2 finished in 14917.04212641716 seconds


<ipython-input-15-77b8f4585760>:67: UserWarning: population /home/z/Dropbox (OIST)/saves/pso_2021-08-25could not be imported
  warn('population ' + pso_name + 'could not be imported',


Generation 3 evaluated. Best fitness: 0.042
Mean fitness = 0.084
to replace: [70, 13, 81, 103, 106, 116, 11, 83, 79, 57, 102, 8, 100, 115, 78, 111, 74, 112, 35, 84, 104, 107, 54, 89, 4, 77, 44, 20, 16, 109]
to mutate: [62, 76, 58, 114, 96, 100, 93, 89]
generation 3 finished in 15410.31172990799 seconds


---
# Particle Swarm Optimization
---

In [20]:
# Utility functions for the PSO algorithm
# These methods assume that 'par_list' and 'ranges' have been defined.
ref_cfg = pop[0] # a configuration with all values, including those that
                 # are not in par_list
    
def add_to_cfg(cfg, vel):
    """ Add a vector to the values of a configuration.
    
        Args:
            cfg : a configuration dictionary
            vel : Numpy array of values to be added
            
        The values in 'vel' are in the order of 'par_list'.
        Each entry in vel is a value in the [-mv, mv] interval, where mv
        is a parameter indicating the maximum velocity.
        
        For each parameter 'par_name' in 'par_list', the corresponding
        value in vel will be multiplied times
        (ranges[par_name]['high'] - ranges[par_name]['low'])
        before being added.
        
        If a value exceeds the limits set in 'ranges' it will be clipped.
        After updating cfg, its 'fitness' and 'n_evals' entries will be
        reset to their initial values.
    """
    for idx, name in enumerate(par_list):
        cfg[name] += vel[idx] * (
                       ranges[name]['high'] - ranges[name]['low'])
        cfg[name] = max( min(ranges[name]['high'], cfg[name]),
                           ranges[name]['low'])
    cfg['fitness'] = None
    cfg['n_evals'] = 0
        
def add_vel(vel, acc, mv=0.5):
    """ Add an acceleration to a velocity vector.
    
        Args:
            vel : velocity vector (numpy array).
            acc : acceleration vector (numpy array).
            mv : maximum velocity.
        Returns:
            vel + acc
            
        The entries of 'vel' and 'acc' are in the order of 'par_list'.
        If vel[i] + acc[i] > mv, or vel[i] + acc[i] < -mv, values will be
        clipped.
    """
    return np.maximum(np.minimum(vel + acc, mv), -mv)
    
def cfg_to_vec(cfg):
    """ Convert a configuration dictionary to a vector.
    
        Args:
            cfg: configuration dictionary.
        Returns:
            vec: a numpy array with the values of the configuration.
            
        The order of the values in 'vec' is that of 'par_list'.
        The 'cfg' entries for 'fitness', 'n_evals', 't_pres', and
        'par_heter' will be omitted in 'vec'.
    """
    vec = np.zeros(len(par_list))
    for idx, name in enumerate(par_list):
        vec[idx] = cfg[name]
    return vec

def vec_to_cfg(vec, fitness=None, n_evals=0, t_pres=None, par_heter=0.01):
    """ Convert a vector to a configuration dictionary.
    
        Args:
            vec: array-like with the values for the configuration.
            'fitness', 'n_evals', 't_pres', 'par_heter': values not present
                in 'vec' that will be appendend to the configuration dict.
        Returns:
            cfg: a configuration dictionary with the values in vec.
            
        The order of the values in vec should be that of 'par_list'.
    """
    cfg = ref_cfg
    for val, name in zip(vec, par_list):
        cfg[name] = val
    cfg['fitness'] = fitness
    cfg['n_evals'] = n_evals
    cfg['t_pres'] = t_pres
    cfg['par_heter'] = par_heter
    return cfg


In [21]:
###############################
###### The PSO algorithm ######
###############################

W = 0.3 # inertia weight
c1 = 0.5 # weight of accel towards personal best
c2 = 0.2 # weight of accel towards global best
mv = 0.3 # maximum velocity (relative to range witdth of paramter)

use_gene = False # insert particles from a concomitant genetic algorithm
sig = lambda f: 1./(1. + np.exp(-4.*(f + 0.1))) # to set probability of insertion

max_iters = 10 # maximum number of iterations
n_procs = 10 # number of processes to use for fitness evaluation

g_best_f = 1e10 # best fitness so far
p_best_fs = [1e10] * len(pop) # best personal fitnesses
t_pres = pop[0]['t_pres'] # assuming all presentation times are equal
vels = np.random.random((len(pop), len(par_list))) - 0.5 # initial velocities

g_best = cfg_to_vec(pop[0]) # arbitrary initialization of global best
p_bests = np.zeros((len(pop), len(par_list)))
for idx, cfg in enumerate(pop):
    p_bests[idx,:] = cfg_to_vec(cfg)

# setting name for file where parameters will be stored
#path = "/home/z/projects/draculab/saves/"
path = "/home/z/Dropbox (OIST)/saves/"
fname1 = "pso"
fname2 = datetime.now().strftime('%Y-%m-%d')
fname = path + fname1 + "_" + fname2

for itr in range(max_iters):
    start_time = time.time()
    
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
        ######## parallel version, multiprocessing module
#     with Pool(n_procs) as p:
#         fits = list(p.map(eval_config, pop))
#         p.close()
#         p.join()
    ######## parallel version, pathos
    with ProcessingPool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
    #print(fits)
    # 1.2) update the average fitness values
    # In PSO configurations change every iteration, so n_evals is 
    # either 0 or 1. However, some configuration may have come from the
    # genetic algorithm, so we consider the case n_evals > 0
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        if nr > 0:
            if nr <= max_evals: 
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
        
    # 2) Find indexes that sort according to fitness. Lowest error first.
    srt_idx = np.argsort(fits)
    
    # 3.1) Update g_best
    if fits[srt_idx[0]] < g_best_f:
        g_best_f = fits[srt_idx[0]]
        g_best = cfg_to_vec(pop[srt_idx[0]])
        
    for idx, cfg in enumerate(pop):
        cfg_vec = cfg_to_vec(cfg)
        # 3.2) Update p_bests
        if cfg['fitness'] < p_best_fs[idx]:
            p_best_fs[idx] = cfg['fitness']
            p_bests[idx] = cfg_vec
            
        # 4) Update velocities
        r1, r2 = np.random.random(2)
        vels[idx] = add_vel(W * vels[idx], c1 * r1 * (p_bests[idx] - cfg_vec))
        vels[idx] = add_vel(vels[idx], c2 * r2 * (g_best - cfg_vec))
    
    # 5) Save current iteration
    best_pop = [vec_to_cfg(g_best, fitness=g_best_f, n_evals=1, t_pres=t_pres)] + pop
    with open(fname, 'wb') as f:
        pickle.dump(best_pop, f)
        f.close()
    
    # A quick message
    print("Iteration %d evaluated. Best fitness: %.3f"%(itr, g_best_f))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if g_best_f < target_fitness:
        print("Good enough parameters found. Stopping search.")
        break
    
    # 6) Update particles
    for idx, cfg in enumerate(pop):
        add_to_cfg(cfg, vels[idx])
        
    #---------------------------------------------------------------------------------
    # 7) Inserting a configuration from the genetic algorithm
    if use_gene:
        g_name = path + "gene_" + fname2
        try:
            with open(g_name, 'rb') as f:
                gene_pop = pickle.load(f)
                f.close()
            gene_cfg = gene_pop[0]
            in_prob = sig(g_best_f - gene_cfg['fitness']) # insertion probability
            if np.random.random() < in_prob:
                rem_idx = np.random.randint(len(pop))
                pop[rem_idx] = gene_cfg
                fits[rem_idx] = gene_cfg['fitness']
                print("inserted pop " + str(rem_idx) + "!!!")    
        except IOError:
            if itr > 2: # if the genetic algorithm should likely be done
                from warnings import warn
                warn('population ' + g_name + 'could not be imported',
                     UserWarning)
                
    print('Iteration %d finished in %s seconds' % (itr, time.time() - start_time))
            

Iteration 0 evaluated. Best fitness: 0.036
Mean fitness = 0.042
Iteration 0 finished in 2070.7380545139313 seconds
Iteration 1 evaluated. Best fitness: 0.036
Mean fitness = 0.293
Iteration 1 finished in 7464.623072147369 seconds
Iteration 2 evaluated. Best fitness: 0.036
Mean fitness = 0.223
Iteration 2 finished in 9718.175866365433 seconds
Iteration 3 evaluated. Best fitness: 0.034
Mean fitness = 0.136
Iteration 3 finished in 9253.007189273834 seconds
Iteration 4 evaluated. Best fitness: 0.034
Mean fitness = 0.188
Iteration 4 finished in 9990.174317598343 seconds
Iteration 5 evaluated. Best fitness: 0.034
Mean fitness = 0.113
Iteration 5 finished in 9394.701133012772 seconds
Iteration 6 evaluated. Best fitness: 0.034
Mean fitness = 0.155
Iteration 6 finished in 9741.45434832573 seconds
Iteration 7 evaluated. Best fitness: 0.034
Mean fitness = 0.133
Iteration 7 finished in 9645.113165140152 seconds
Iteration 8 evaluated. Best fitness: 0.032
Mean fitness = 0.132
Iteration 8 finished in 

In [22]:
# print final population
for dic in pop[:5]:
    print('{',end='')
    for name in dic.keys():
        if name != 'fitness' or dic['fitness'] != None:
            print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'A__M_lrate':17.05, 'A__M_w_max_frac':0.29, 'A__M_w_sum':1.00, 'AL_thresh':0.53, 'b_e':0.50, 'C__C_antag':1.46, 'C__C_p_antag':0.26, 'C__C_p_syne':0.28, 'C__C_syne':1.00, 'C_adapt_amp':7.90, 'C_cid':0.15, 'C_sigma':0.52, 'C_slope':1.79, 'C_tau':0.22, 'C_tau_slow':40.00, 'C_thresh':0.75, 'CE__CI_w':0.52, 'CI__CE_w':-1.32, 'g_e_03':23.64, 'CI_slope':3.58, 'CI_tau':0.02, 'CI_thresh':1.42, 'g_e_factor':3.28, 'II_g_03':6.67, 'M__C_lrate':453.11, 'M__C_w_sum':0.50, 'M__M_w':0.00, 'M_cid':1.09, 'M_des_out_w_abs_sum':2.95, 'M_tau':0.02, 'SF_thresh_03':0.63, 'SPF__SPF_w':-1.50, 'n_evals':0.00, 't_pres':40.00, 'par_heter':0.01, }

{'A__M_w_max_frac':0.29, 'A__M_w_sum':1.00, 'C_adapt_amp':0.00, 'C_cid':0.14, 'C_sigma':0.53, 'M_cid':1.10, 'M_des_out_w_abs_sum':3.10, 'g_e_factor':3.18, 'C_slope':2.19, 'C_thresh':0.95, 'C_tau':0.24, 'C_tau_slow':21.00, 'A__M_lrate':19.90, 'AL_thresh':0.55, 'b_e':0.50, 'C__C_antag':1.50, 'C__C_p_antag':0.25, 'C__C_p_syne':0.28, 'C__C_syne':1.00, 'CE__CI_w':0.50, 'CI

{'A__M_lrate': 17.04596842109897,
 'A__M_w_max_frac': 0.2886964823653161,
 'A__M_w_sum': 1.0,
 'AL_thresh': 0.5272262191174888,
 'b_e': 0.5,
 'C__C_antag': 1.463225865210005,
 'C__C_p_antag': 0.2561389342079896,
 'C__C_p_syne': 0.28040786488117775,
 'C__C_syne': 0.9971516811172368,
 'C_adapt_amp': 7.90081142557846,
 'C_cid': 0.1469253648295994,
 'C_sigma': 0.5197905018652487,
 'C_slope': 1.792937009255434,
 'C_tau': 0.2176344558450552,
 'C_tau_slow': 40.0,
 'C_thresh': 0.745060076191622,
 'CE__CI_w': 0.5183448406195807,
 'CI__CE_w': -1.3183518422723193,
 'g_e_03': 23.635859645594188,
 'CI_slope': 3.5841010228235923,
 'CI_tau': 0.01628753477924452,
 'CI_thresh': 1.4170961707059726,
 'g_e_factor': 3.2848626020247464,
 'II_g_03': 6.665933092061302,
 'M__C_lrate': 453.1124675492799,
 'M__C_w_sum': 0.5,
 'M__M_w': 0.0,
 'M_cid': 1.0923934596194838,
 'M_des_out_w_abs_sum': 2.9500457512263565,
 'M_tau': 0.022296085722734196,
 'SF_thresh_03': 0.6268491900130222,
 'SPF__SPF_w': -1.4982115204290

In [23]:
fname

'/home/z/Dropbox (OIST)/saves/pso_2021-06-28'

In [ ]:
# Save the final population
fname = "v3_nst_afx_pop"
fname += "_" + datetime.now().strftime('%Y-%m-%d__%H_%M')
with open(fname, 'wb') as f:
    pickle.dump(pop, f)
    f.close()

In [ ]:
# Load a saved population
import pickle
fname = 'v3_nst_afx_pop'
with (open(fname, "rb")) as f:
    results = pickle.load(f)
    f.close()

In [ ]:
# Test a configuation
cfg_id = 0 # index in the population for the configuration
net, pops_dict, hand_coords, m_idxs, t_pres, _  = net_from_cfg(pop[cfg_id])
pops_names = ['SF', 'SP', 'SPF', 'AL', 'AF', 'SP_CHG', 'CE', 'CI', 'M', 'ACT', 'P']
for name in pops_names:
    exec("%s = %s"% (name, str(pops_dict[name])))

start_time = time.time()
times, data, plant_data  = net.flat_run(600.)
#times, data, plant_data  = net.run(40.)
print('Execution time is %s seconds' % (time.time() - start_time))
data = np.array(data)
#Execution time is 8.687349319458008 seconds  << before sc_inp_sum_mp, flat_run(5.)

Same history as `v3_nst_afx`
...
